# Generating lattice paths for any lattice polytope $\Delta$

In this notebook, we will write functions that compute all lattice paths in a lattice polygon $\Delta$  and determine their multiplicity. Define a total order on $\mathbb{Z}^2$: 
\begin{equation}
    (x_1,y_1) \leq (x_2,y_2) \hspace{10pt} \text{ if } \hspace{20pt} x_1<x_2 \hspace{15pt} \text{ or } \hspace{15pt}  x_1 = x_2 \text{ and } y_1\geq y_2
\end{equation}
A lattice path $\gamma$ is *increasing* if its increasing with respect to this ordering. Let $u_{\Delta}$ be the initial lattice point in $\Delta$ and $v_{\Delta}$ the terminal. Let $\vec{\beta} = \{\beta_{e} \, : \, e \text{ is a face of } \Delta\}$ be a collection of sequences indexed by the faces of $\Delta$. The sequence $\beta_{e} = (\beta_e(1), \beta_e(2),\ldots)$ consists of nonnegative integers, and
 \begin{equation}
      I\beta_{e} = \sum_{k\geq 1} k\cdot \beta_{e}(k) \hspace{20pt} |\beta_{e}| = \sum_{k\geq 1} \beta_{e}(k) \hspace{15pt} |\vec{\beta}| = \sum_{e\in E(\Delta)} |\beta_{e}|
\end{equation}
Given a genus $g$ and lattice polygon $\Delta$, and $\vec{\beta} = \{\beta_{e} : e\in E(\Delta)\}$, we wish to generate all increasing lattice paths with $|\vec{\beta}| + g - 1$ steps from $u_{\Delta}$ to $v_{\Delta}$ that have nonzero multiplicity, which is defined in the following way. 

Let $\gamma_{-}(\Delta)$  be the unique path in $\mathbb{R}^2$ from $u_{\Delta}$ to $v_{\Delta}$ contained in $\partial \Delta$ that goes below $\Delta$, and $\gamma_{+}(\Delta)$ the unique such path that goes above $\Delta$. 

Now let $\gamma$ be any lattice path as above. The **negative multiplicity $\operatorname{mult}_{-}(\gamma)$** satisfies the recursive formula:

\begin{equation*}
  \operatorname{mult}_{-}(\gamma) = \text{Area}(P) \cdot \operatorname{mult}_{-}(\gamma') + \operatorname{mult}_{-}(\gamma'')  
\end{equation*}

where $\gamma'$ is obtained from $\gamma$ by "cutting the corner" at the first *right* turn, $\gamma''$ is obtained from $\gamma$ by completing the parallelogram $P$ at this turn. It satisfies the initial data:

<ul>
  <li> $\operatorname{mult}_{-}(\gamma) = 1$ for any path $\gamma$ counterclockwise along $\partial \Delta$, and along the face $e$ has $\beta_{e}(i)$ steps of size $i$; </li>
  <li> $\operatorname{mult}_{-}(\gamma) = 0$ whevever $\gamma$ leaves $\Delta$ or goes counterclockwise along $\partial \Delta$ but does not have the required step sizes prescribed by $\vec{\beta}$.  </li>
</ul> 

The **positive multiplicity** $\operatorname{mult}_{+}(\gamma)$ satisfies the recursive formula:

\begin{equation*}
  \operatorname{mult}_{+}(\gamma) = \text{Area}(P) \cdot \operatorname{mult}_{+}(\gamma') + \operatorname{mult}_{+}(\gamma'')  
\end{equation*}

where $\gamma'$ is obtained from $\gamma$ by "cutting the corner" at the first *left* turn, $\gamma''$ is obtained from $\gamma$ by completing the parallelogram $P$ at this turn. It satisfies the initial data:

<ul>
    <li> $\operatorname{mult}_{+}(\gamma) = 1$ for any path $\gamma$ clockwise along $\partial \Delta$, and along the face $e$ has $\beta_{e}(i)$ steps of size $i$; </li>
  <li> $\operatorname{mult}_{+}(\gamma) = 0$ whevever $\gamma$ leaves $\Delta$ or goes clockwise along $\partial \Delta$ but does not have the required step sizes prescribed by $\vec{\beta}$.  </li>
</ul> 

The **multiplicity** of $\gamma$ is
\begin{equation}
\operatorname{mult}(\gamma) = \operatorname{mult}_{-}(\gamma) \cdot \operatorname{mult}_{+}(\gamma)
\end{equation}

Compare to 
<a href="https://arxiv.org/pdf/math/0504392.pdf">The Caporaso-Harris formula and plane relative Gromov-Witten invariants in tropical geometry</a> by A. Gathmann and H. Markwig. 

Throughout, a lattice path of length $n$ is always represented as a $n \times 2$ matrix, where each row represents a lattice point. When we say that the rows are in increasing order, we will always refer to the total order of $\mathbb{Z}^2$ above. The $\vec{\beta}$ data is recorded as a `Map` whose keys are the faces of $\Delta$ (as a pair of vertices in increasing order) and the value on a face $e$ is $\beta_e$. 

<ul>
    
  <li> <code>pathsDelta</code>. This takes in a genus $g$ and lattice polytope $\Delta$, and generates all increasing lattice paths from $u_{\Delta}$  to $v_{\Delta}$ consisting of $|\vec{\beta}| + g - 1$ steps.   </li>
    
  <li>  <code>initialCacheNegative</code>. This function initiates the recursive algorithm used to compute the negative multiplicity. It is a <code>Map</code> whose keys are the the paths going counterclockwise along $\partial \Delta$ with step sizes governed by $\vec{\beta}$, and the value on each path is 1. </li>

  <li>  <code>initialCachePositive</code>. This function initiates the recursive algorithm used to compute the positive multiplicity. It is a <code>Map</code> whose keys are the the paths going clockwise along $\partial \Delta$ with step sizes governed by $\vec{\beta}$, and the value on each path is 1. </li>
    
  <li>  <code>negMultiplicity</code>. This function computes $\operatorname{mult}_{-}(\gamma)$ using the above recursive algorithm. </li>
    
  <li>  <code>posMultiplicity</code>.  This function computes $\operatorname{mult}_{+}(\gamma)$ using the above recursive algorithm. </li>
  
  <li>  <code>multiplicity</code>.  This function computes $\operatorname{mult}(\gamma)$. </li>

  <li>  <code>nonzero_paths</code>.  This function determines all increasing lattice paths from $u_{\Delta}$ to $v_{\Delta}$ with $|\vec{\beta}| + g - 1$ steps that have nonzero multiplicity. It is a <code>Map</code> whose value on a lattice path is its multiplicity.  </li>
  
    
</ul> 



In [1]:
use strict;
use warnings;
use application "polytope";
use List::Util qw(all none any);
use Data::Dumper qw(Dumper);

---
**Function** `Ia`.

   *Input*: A positive `Integer` $n$. 

   *Output*: The $1\times n$ `Matrix` $[1 \; 2\; \cdots \; n]$. 
   
---

**Function** `sum` (by Lars).

   *Input*: A `Vector` $v$. 

   *Output*: The sum of the entries of $v$. 

---

**Function** `prune_zeros`

*Input*: A `Vector` $v$. 

*Output*: A `Vector` $v'$ obtained from $v$ by removing all trailing $0$'s at the end. 

*Example*: This function takes $v=(1,0,2,3,0,0,0)$ to $v'=(1,0,2,3)$.

---

**Function** `turnData`

*Input*: `$v1, $v2, $v3` each a `Vector` of dimension 2. 

*Output*: `Rational`.

*Description*: returns the cross-product: `($v2-$v1)` x `($v3-$v2) ` where these vectors are viewed in $\mathbb{R}^3$ as the first 2 coordinates.

---

**Function** `negatey`

*Input*: `$Delta = Polytope` of dimension 2. 

*Output*: `Polytope `.

*Description*: returns the polygon obtained by reflecting `$Delta` through the x-axis. 

---

**Function** `negateyLP`

*Input*: `$DeltaLPs = Matrix`. 

*Output*: `Matrix `.

*Description*: Given `$DeltaLPs` a `Matrix` consisting of the lattice points of a polygon, returns a new `Matrix` obtained by negating the last column. Thus, if `$DeltaLPs = $Delta->BOUNDARY_LATTICE_POINTS`, then the rows of `negateyLP($Delta)` coincide with  `negatey($Delta)->BOUNDARY_LATTICE_POINTS`, but in the order as described above. 

---

**Function** `pathsDelta`

*Input*: `$g = Int, $Delta = Polytope, $absbetas = Int`. 

*Output*: `Array<Matrix> `.

*Description*: Given a lattice polytope `$Delta`, genus `$g`, and the number `$absbeta` representing $|\vec{\beta}|$, this function generates all increasing lattice paths from $u_{\Delta}$ to $v_{\Delta}$ consisting of $|\vec{\beta}| + g - 1$ steps. Each lattice path is recorded as a matrix with $2$ columns, where each row is a single lattice point, and the rows are in increasing order. 

---




In [2]:
sub Ia {
    my ($n) = @_;
    my $M = new Matrix(1,$n);
    for (my $i=0; $i<$n; $i++){
        $M -> elem(0,$i) = $i+1;
    }
 return $M   
}

sub sum {
   my $v = shift;
   my $result = 0;
   foreach my $entry (@$v) { $result += $entry; }
   return $result;
}

sub prune_zeros {
   my ($v) = @_;
   my $result = new Vector($v);
   my $i = $result->dim;
   while($i > 0 && $result->[$i-1] == 0){
      $i--;
   }
   return $result->slice(sequence(0, $i));
}


sub turnData {
    my ($v1, $v2, $v3) = @_;
    my $u1 = $v2-$v1; $u2 = $v3-$v2;
    return ($u1->[0])*($u2->[1]) - ($u1->[1])*($u2->[0]);
    }
    
    
sub pathLength {
    my ($g, $absbetas) = @_;
    return $absbetas + $g - 1;
}


sub negatey {
    my ($Delta) = @_;
    my $negy = new Matrix([[1,0,0],[0,1,0],[0,0,-1]]);
    return transform($Delta, $negy);
}


sub negateyLP {
    my ($DeltaLPs) = @_;
    my $negy = new Matrix([[1,0,0],[0,1,0],[0,0,-1]]);
    return $DeltaLPs * $negy;
}


sub pathsDelta {
    my ($g, $Delta, $absbetas) = @_;
    
    my $negyDelta = negatey($Delta);
    my $negyDeltaLPts = $negyDelta->LATTICE_POINTS;
    my $DeltaLPts = negateyLP($negyDeltaLPts) -> minor(All, ~[0]);
    
    my $nLPts = $DeltaLPts -> rows;     
    my $u = $DeltaLPts -> row(0); 
    my $v = $DeltaLPts -> row($nLPts-1); 
    
    my $length = pathLength($g, $absbetas);  
    my $options = all_subsets_of_k(range(1, $nLPts-2), $length-1); 
    my $options_arr = new Array<Set>(@$options); 
    
    my $optionsuv = new Array<Matrix>($options->size); 
    
    for (my $i = 0; $i < $options->size; $i++) {
        my $optioni = new Matrix($DeltaLPts -> minor(new Set([0, $nLPts-1]) + $options_arr->[$i ], All));
        $optionsuv ->[$i ] = $optioni;
    }
    
    return $optionsuv;
}


---

**Function** `sortLPts`

*Input*: `$LPts = Matrix`. 

*Output*: `Matrix `.

*Description*: The matrix `$LPts` has 2 columns, and each row represents a lattice point in $\mathbb{R}^2$. This function returns a new `Matrix` of the same type, but the rows are in increasing order. 

---



In [3]:
sub sortLPts {
    my ($LPts) = @_;
    my $nLPts = $LPts -> rows;
        
    my @rows = ();
    for (my $i = 0; $i< $LPts -> rows; $i++){
        push(@rows, $LPts->[$i ]);
    }

    @rows_sort =  sort { $a->[0] <=> $b->[0] || $b->[1] <=> $a->[1] } @rows;
    
    my $sortedLPts = new Matrix(@rows_sort);
    return $sortedLPts;

}

---

**Function** `set2Beta`.

*Input*: `$set = Set<Rational>`. 

*Output*: `Vector<Rational>`.

*Description*: Given a subset of $\{0,\ldots,n\}$, returns the $\beta = (\beta(1),\ldots,\beta(k))$ vector of differences. That is, $\beta(i)$ is the number of steps of size $i$. 

*Example* On the set $\{1,2,4,5,7,11\}$, this function returns $\beta = (2,2,0,1)$.

---

**Function** `setsGivenBeta`.

*Input*: `$beta = Vector<Rational>`. 

*Output*: `Array<Set>`.

*Description*: Given a $\beta$ vector, this function returns all subsets of $\{0,\ldots,I\beta\}$  whose vector of differences (as in `set2Beta`) is $\beta$. 

---

**Function** `edgeLPGivenBeta`. 

*Input*: `$beta = Vector<Rational>, $start = Vector<Rational>, $end = Vector<Rational>`. 

*Output*: `Array<Matrix>`.

*Description*: Given $\beta = (\beta(1),\ldots,\beta(k))$  and 2 points `$start, $end` in $\mathbb{Z}$, this function returns all lattice paths from `$start` to `$end` with $\beta(i)$ steps of lattice-length $i$.

---

**Function** `aboveOrBelowuv_SingleVertex`. 

*Input*: `$u = Vector<Rational>, $v = Vector<Rational>, $w = Vector<Rational>`. 

*Output*: `Int`.

*Description*: The vectors `$u` and `$v` represent the initial and terminal vertices of a lattice polygon on $\mathbb{R}^2$. This function returns $1$ if `$w` lies above the line spanned by `$u` and `$v`, $-1$ if it lies below, and $0$ if it lies on this line.

---

**Function** `aboveOrBelowuv_AllVertex`. 

*Input*: `$u = Vector<Rational>, $v = Vector<Rational>, $verts = Array<Vector<Rational>>`. 

*Output*: `Map<Vector,Int>`.

*Description*: The keys of this `Map` are the vectors in `$verts`, and the value on the vector `$w` is `aboveOrBelowuv_SingleVertex($u,$v,$w)`.

---

**Function** `lowerUpperVertices`. 

*Input*: `$u = Vector<Rational>, $v = Vector<Rational>, $verts = Array<Vector<Rational>>`. 

*Output*: `Pair<Matrix,Matrix>`.

*Description*: These matrices have 2 colomns each, and the rows consist of the vertices of a lattice polygon $\Delta$, in increasing order. The first matrix consists of the vertices lying on $\partial \Delta$ going counterclockwise from `$u` to `$v`,  and the second matrix consists of the vertices lying on $\partial \Delta$ going clockwise from `$u` to `$v`. 

---

**Function** `lowerUpperFaces`.

*Input*: `$u = Vector<Rational>, $v = Vector<Rational>, $verts = Array<Vector<Rational>>`. 

*Output*: `Pair< Array<Pair<Vector,Vector>>, Array<Pair<Vector,Vector>> >`.

*Description*: These arrays consist of the faces of a lattice polygon $\Delta$, recorded as a pair of vertices in increasing order. The first array consists of the faces going counterclockwise from `$u` to `$v`, and the second array consists of the faces going clockwise from `$u` to `$v`. 


---

**Function** `join2SetOfPaths_Unordered`. 

*Input*: `$p1 = Array<Matrix>, $p2 = Array<Matrix>`. 

*Output*: `Array<Matrix>`.

*Description*: Each matrix in `$p1` and `$p2` has 2 columns, representing a lattice path in $\mathbb{R}^2$ (possibly redundant and unordered). This function returns all paths obtained from appending a path in `$p2` to a path in `$p1`. 

---
**Function** `initialCacheNegative`.

*Input*: `$Delta = Polytope, $betas = Map<Pair<Vector,Vector>, Vector>`. 

*Output*: `Map<Matrix, Rational>`.

*Description*: The map `$betas` records the $\vec{\beta}$-vector for each face. Its keys are the faces of $\Delta$ as a pair of vectors in increasing order. The value on such a vector is its $\beta$-vector. This function creates a `Map` whose keys are the paths that go counterclockwise along $\partial\Delta$ having step-sizes governed by the $\vec{\beta}$-vectors. The value on each such path is 1. 

---
**Function** `initialCachePositive`. 

*Input*: `$Delta = Polytope, $betas = Map<Pair<Vector,Vector>, Vector>`. 

*Output*: `Map<Matrix, Rational>`.

*Description*: The map `$betas` records the $\vec{\beta}$-vector for each face. Its keys are the faces of $\Delta$ as a pair of vectors in increasing order. The value on such a vector is its $\beta$-vector. This function creates a map whose keys are the  paths that go clockwise along $\partial\Delta$ having step-sizes governed by the $\vec{\beta}$-vectors. The value on each such path is 1. 



In [4]:
sub set2Beta {
    my ($set) = @_;
    
    my $set_arr = new Array<Rational>($set); 
    my $k = $set_arr->size; 
   
    my $differences = new Array<Int>($k-1);
    for (my $i = 0; $i < $k-1; $i++) {
        $differences -> [$i ] = convert_to<Int>($set_arr -> [$i+1] - $set_arr -> [$i ]);    
    }
    
    
    my $max = maximum($differences); 
    my $betaFromSet = new Vector<Rational>($max);
    
    for (my $j = 0; $j < $k-1; $j++){
        $betaFromSet -> [$differences -> [$j ] - 1] += 1;  
    }
    return $betaFromSet;
}

sub setsGivenBeta {
    my ($beta) = @_;
    
    my $absbeta = sum($beta); 
    
    if ($absbeta == 0) {return new Array<Set>(new Set([0]))}
    
    my $Ibeta = convert_to<Int>((Ia($beta->dim) * $beta) -> [0]);
    my $options = new Array<Set>(all_subsets_of_k(range(0, $Ibeta), $absbeta+1));
        
    my @sets = ();
    
    for (my $i=0; $i<$options->size; $i++) {
        my $optioni = $options->[$i ];
        my $optionbeta = set2Beta($optioni);
        if (prune_zeros($optionbeta) == prune_zeros($beta) ) {
            push(@sets, $optioni); 
        } 
    }
    
    if (scalar @sets == 1) {return new Array<Set>([$sets[0]])}
    
    return new Array<Set>(@sets);

}

sub edgeLPGivenBeta {
    my ($beta, $start, $end) = @_;
    
    $start = new Vector((1,@$start)); 
    $end = new Vector((1,@$end)) ;
    
    my $segment = new Polytope(POINTS => [$start, $end ]); 
    my $segmentLP = $segment -> LATTICE_POINTS;
    $segmentLP = sortLPts($segmentLP -> minor(All, ~[0])); 

    my $absbeta = sum($beta);
    my $sets = setsGivenBeta($beta);
    
    my $paths = new Array<Matrix>($sets->size);
    
    my $k=0; 
    
    foreach my $s (@$sets) { 
        
        my $p = new Matrix(convert_to<Int>($absbeta)+1, 2); 
        my $j = 0;
        foreach my $i (@$s) {
            $p -> row($j) = $segmentLP -> row($i);
            $j++;
        }
        $paths->[$k ] = $p;
        $k++;
    }
    return $paths
}

sub aboveOrBelowuv_SingleVertex {
    my ($u,$v,$w) = @_;
    
    my $v_u = $v - $u;
    my $w_u = $w - $u;
    
    my $A = new Matrix<Rational>([$v_u, $w_u]);
    my $dA = det($A);
    
    if ($dA>0) {return new Int(1)};
    if ($dA==0) {return new Int(0)};
    if ($dA<0) {return new Int(-1)};
    
}

sub aboveOrBelowuv_AllVertices {
    my ($u,$v,$verts) = @_;
    
    my $signs = new Map<Vector,Int>;
    foreach my $w (@$verts) {
        $signs -> {$w} = aboveOrBelowuv_SingleVertex($u,$v,$w);
    }
    return $signs
}

sub lowerUpperVertices {
    my ($u,$v,$verts) = @_;
    my $signs = aboveOrBelowuv_AllVertices($u, $v, $verts);
    my @lowerPath = ();
    my @upperPath = ();
    
    my $allSigns = new Array<Int>(values(%$signs));
    
    
    if (all {$_ ge 0} @$allSigns) {
        push @upperPath, ($u,$v);
        foreach my $w (@$verts) {
            if ($signs->{$w} > 0) {push @upperPath, $w};
            if ($signs->{$w} == 0) {push @lowerPath, $w};            
        }
    }

    if (all {$_ le 0} @$allSigns) {
        push @lowerPath, ($u,$v);
        foreach my $w (@$verts) {
            if ($signs->{$w} < 0) {push @lowerPath, $w};
            if ($signs->{$w} == 0) {push @upperPath, $w};            
        }
    }
    
    if ( (any {$_ > 0} @$allSigns) and (any {$_ < 0} @$allSigns)  )   {
        push @lowerPath, ($u,$v);
        push @upperPath, ($u,$v);
        foreach my $w (@$verts) {
            if ($signs->{$w} < 0) {push @lowerPath, $w};
            if ($signs->{$w} > 0) {push @upperPath, $w};            
        }
    }
    return new Pair<Matrix,Matrix>(sortLPts(new Matrix(@lowerPath)), sortLPts(new Matrix(@upperPath)));
}


sub lowerUpperFaces {

    my ($u,$v,$verts) = @_;

    my $lUV = lowerUpperVertices($u,$v,$verts);
    
    my $lowerVertices = $lUV -> [0];
    my $upperVertices = $lUV -> [1];
    
    my $nL = ($lowerVertices ) -> rows; 
    my $nU = ($upperVertices ) -> rows; 
        
    my $lowerFaces = new Array<Pair<Vector,Vector>>($nL-1);
    my $upperFaces = new Array<Pair<Vector,Vector>>($nU-1);

    for (my $i=0; $i<$nL-1; $i++) {
        $lowerFaces -> [$i] = new Pair<Vector,Vector>($lowerVertices -> [$i ], $lowerVertices -> [$i+1 ]   );    
    }

    for (my $i=0; $i<$nU-1; $i++) {
        $upperFaces -> [$i] = new Pair<Vector,Vector>($upperVertices -> [$i ], $upperVertices -> [$i+1 ]   );    
    }
    return new Pair< Array<Pair<Vector,Vector>>, Array<Pair<Vector,Vector>> >($lowerFaces, $upperFaces)

}

sub join2SetOfPaths_Unordered {
    my ($paths1, $paths2) = @_;
    @newPaths = ();
    foreach my $p1 (@$paths1) {
        foreach my $p2 (@$paths2) {
            push @newPaths, new Matrix([@$p1, @$p2 ]);
        }
    }
    return new Array<Matrix>([@newPaths])
}



sub initialCacheNegative {
    my ($Delta, $betas) = @_;
    
    my $verts = sortLPts(($Delta -> VERTICES) -> minor(All, ~[0]));
    my $u = $verts->row(0); my $v = $verts->row($verts->rows-1);
    my $signs = aboveOrBelowuv_AllVertices($u, $v, $verts); 
    my $cache = new Map<Matrix, Rational>;
    my $lowerUF = lowerUpperFaces($u, $v, $verts);
    my $lowerFaces = $lowerUF -> [0]; 

    my $lowerPaths_unordered = new Array<Matrix>(1);

    foreach my $s (@$lowerFaces) {
        my $newLPs = edgeLPGivenBeta($betas->{$s}, $s->[0], $s->[1]); 
        my $newLowerPaths = join2SetOfPaths_Unordered($lowerPaths_unordered, $newLPs); 
        $lowerPaths_unordered = new Array<Matrix>([@$newLowerPaths]); 
    }
    
    foreach my $p (@$lowerPaths_unordered) {
        #print "p=$p\n";
        my $uniquep = new Matrix(new Set<Vector>(@$p));        
        $cache -> {sortLPts($uniquep)} = 1;
    }
    
    return $cache;
}


sub initialCachePositive {
    my ($Delta, $betas) = @_;
    
    my $verts = sortLPts(($Delta -> VERTICES) -> minor(All, ~[0])); 
    my $u = $verts->row(0); my $v = $verts->row($verts->rows-1); 
    my $signs = aboveOrBelowuv_AllVertices($u, $v, $verts); 
    my $cache = new Map<Matrix, Rational>;
    my $lowerUF = lowerUpperFaces($u, $v, $verts);
    my $upperFaces = $lowerUF -> [1]; 

    my $upperPaths_unordered = new Array<Matrix>(1);

    foreach my $s (@$upperFaces) {
        
        my $newLPs = edgeLPGivenBeta($betas->{$s}, $s->[0], $s->[1]);         
        my $newUpperPaths = join2SetOfPaths_Unordered($upperPaths_unordered, $newLPs); 
        $upperPaths_unordered = new Array<Matrix>([@$newUpperPaths]); 
    }
    
    foreach my $p (@$upperPaths_unordered) {
        #print "p=$p\n";
        my $uniquep = new Matrix(new Set<Vector>(@$p));        
        $cache -> {sortLPts($uniquep)} = 1;
    }
    
    return $cache;
}

**Function** `negMultiplicity`

*Input*: `$g = Int, $Delta = Polytope, $LP = Matrix, $reclevel = Int`. 

*Output*: `Rational`.

*Description*: This function computes the negative multiplicity $\mu_{-}$ of the lattice path `$LP`. 

---

**Function** `negMultiplicity_recursive`

*Input*: `$g = Int, $Delta = Polytope, $LP = Matrix, $reclevel = Int`. 

*Output*: `Rational`.

*Description*: The negative multiplicity is computed via a recursive algorithm. This function implements the recursion.  

---


In [5]:
sub negMultiplicity {
    my ($g, $Delta, $betas, $LP, $reclevel, $makeCache) = @_;
    
    if (defined($makeCache) && $makeCache) {$cacheNegative = initialCacheNegative($Delta, $betas)} 
    
    if(!defined($cacheNegative->{$LP})){
        my $val = negMultiplicity_recursive($g, $Delta, $betas, $LP, $reclevel);
        $cacheNegative->{$LP} = $val;
    }
    
    return $cacheNegative->{$LP};
}

sub negMultiplicity_recursive {
    my ($g, $Delta, $betas, $LP, $reclevel) = @_;
    
    my $space = "| " x $reclevel;
    
    my $n = $LP->rows;
    
    my $LP_arr = new Array<Vector>(@$LP);
    
    for (my $i=1; $i<$n-1; $i++){
        my $turn = turnData($LP_arr->[$i-1], $LP_arr->[$i ], $LP_arr->[$i+1]); 
 
        if ($turn < 0) {
            my $LP1 = $LP -> minor(~[$i ], All); 
     
            my $muLP1 = negMultiplicity($g, $Delta, $betas, $LP1, $reclevel+1); 
            my $muLP2 = 0;
            
            my $newLPoint = $LP_arr->[$i-1] + ($LP_arr->[$i+1] - $LP_arr->[$i ]);
            
            if ($Delta->contains(new Vector((1), @$newLPoint))) {

                my $LP2 = new Matrix($LP);
                $LP2 -> row($i) = $newLPoint; 
                $muLP2 = negMultiplicity($g, $Delta, $betas, $LP2, $reclevel+1);
            }
            return -$turn * $muLP1 + $muLP2;
        }
    }
    return 0
}


**Function** `posMultiplicity`

*Input*: `$g = Int, $Delta = Polytope, $LP = Matrix, $reclevel = Int`. 

*Output*: `Rational`.

*Description*: This function computes the positive multiplicity $\mu_{+}$ of the lattice path `$LP`. 

---

**Function** `posMultiplicity_recursive`

*Input*: `$g = Int, $Delta = Polytope, $LP = Matrix, $reclevel = Int`. 

*Output*: `Rational`.

*Description*: The positive multiplicity is computed via a recursive algorithm. This function implements the recursion.  

---

In [6]:
sub posMultiplicity {
    my ($g, $Delta, $betas, $LP, $reclevel, $makeCache) = @_;
    
    if (defined($makeCache) && $makeCache) {$cachePositive = initialCachePositive($Delta, $betas)} 
    
    if(!defined($cachePositive->{$LP})){
        my $val = posMultiplicity_recursive($g, $Delta, $betas, $LP, $reclevel);
        $cachePositive->{$LP} = $val;
    }
    
    return $cachePositive->{$LP};
}

sub posMultiplicity_recursive {
    my ($g, $Delta, $betas, $LP, $reclevel) = @_;
    
    my $space = "| " x $reclevel;
    
    my $n = $LP->rows; 
    
    my $LP_arr = new Array<Vector>(@$LP);
    
    for (my $i=1; $i<$n-1; $i++){
        my $turn = turnData($LP_arr->[$i-1], $LP_arr->[$i ], $LP_arr->[$i+1]); 
 
        if ($turn > 0) {
            my $LP1 = $LP -> minor(~[$i ], All); 
            
        
            my $muLP1 = posMultiplicity($g, $Delta, $betas, $LP1, $reclevel+1); 
            my $muLP2 = 0;
            
            my $newLPoint = $LP_arr->[$i-1] + ($LP_arr->[$i+1] - $LP_arr->[$i ]);
            
            if ($Delta->contains(new Vector((1), @$newLPoint))) {

                my $LP2 = new Matrix($LP);
                $LP2 -> row($i) = $newLPoint; 
                $muLP2 = posMultiplicity($g, $Delta, $betas, $LP2, $reclevel+1);
            }
            return $turn * $muLP1 + $muLP2;
        }
    }
    return 0
}

In [7]:
sub sortedBeta{
    my ($betas) = @_;
    
    my $newbetas = new Map<Pair<Vector,Vector>, Vector>;
    
    foreach my $beta (keys %$betas) {
    
        my $betaMatrix = new Matrix($beta->[0], $beta->[1]); 
        my $sortedBetaMatrix = sortLPts($betaMatrix); 
        my $sortedbeta = new Pair<Vector, Vector>($sortedBetaMatrix->row(0), $sortedBetaMatrix->row(1)); 
    
        $newbetas -> {$sortedbeta} = $betas -> {$beta}; 
    
    }
    return $newbetas;
}
    

**Function** `multiplicity`

*Input*: `$g = Int, $Delta = Polytope, $LP = Matrix`. 

*Output*: `Rational`.

*Description*: This function computes the multiplicity $\operatorname{mult}$ of the lattice path `$LP`. 

---

**Function** `count_lattice_paths_with_multiplicity`

*Input*: `$g = Int, $Delta = Polytope`. 

*Output*: `Rational`.

*Description*: This function computes the sum of the multiplicities of all lattice paths from $u_{\Delta}$ to $v_{\Delta}$ of length $|\vec{\beta}| +g - 1$. 

---
**Function** `nonzero_paths`

*Input*: `$g = Int, $Delta = Polytope`. 

*Output*: `Map<Matrix, Rational>`.

*Description*: This function returns a dictionary (as a `Map`) whose keys are the lattice paths in $\Delta$ from $u_{\Delta}$ to $v_{\Delta}$ of length $|\vec{\beta}| +g - 1$ and nonzero multiplicity. On such a lattice path, the value is its multiplicity.  

---


In [8]:
sub multiplicity {
    my ($g, $Delta, $betas, $LP, $makeCache) = @_;
    
    my $sortedBetas = sortedBeta($betas);
    
    if (defined $makeCache && $makeCache) {
        $cachePositive = initialCachePositive($Delta, $sortedBetas);
        $cacheNegative = initialCacheNegative($Delta, $sortedBetas);
    }
    
    return negMultiplicity($g, $Delta, $sortedBetas, $LP) * posMultiplicity($g, $Delta, $sortedBetas, $LP);
}


sub numberEnds {
    my ($betas) = @_;
    $result = 0; 
    $betas_arr = new Array<Vector>(values(%$betas));
    for (my $i=0; $i<$betas_arr->size; $i++) {
        $result += sum($betas_arr->[$i ]);    
    }
    return $result;
}


sub count_lattice_paths_with_multiplicity {
    my ($g, $Delta, $betas) = @_;
    
    my $absbetas = numberEnds($betas);
    my $paths = pathsDelta($g, $Delta, $absbetas); 
    my $multiplicities = new Vector($paths->size);
    
    my $sortedBetas = sortedBeta($betas);

    
    $cacheNegative = initialCacheNegative($Delta, $sortedBetas); 
    $cachePositive = initialCachePositive($Delta, $sortedBetas); 
    
    
    
    for (my $i=0; $i<$paths->size; $i++) {
        my $LP = $paths->[$i ]; 
                
        my $multLP = multiplicity($g, $Delta, $sortedBetas, $LP); 
        $multiplicities -> [$i ] = $multLP;
    }
    
    return sum($multiplicities)
}


sub nonzero_paths {
    my ($g, $Delta, $betas ) = @_;
    
    my $absbetas = numberEnds($betas);
    my $paths = pathsDelta($g, $Delta, $absbetas);  
    my $multiplicities = new Map<Matrix, Rational>; 
    
    my $sortedBetas = sortedBeta($betas);
    
    $cachePositive = initialCachePositive($Delta, $sortedBetas); 
    $cacheNegative = initialCacheNegative($Delta, $sortedBetas); 
    
    
    for (my $i=0; $i<$paths->size; $i++) {
        my $LP = $paths->[$i ]; 
        my $multLP = multiplicity($g, $Delta, $sortedBetas, $LP); 
        if ($multLP!=0) {$multiplicities -> {$LP} = $multLP};
         
    }
    return $multiplicities
    
}

In [9]:
$Delta = new Polytope(POINTS=>[[1,0,0], [1,3,0], [1,0,5], [1,3,5]]);
$vertices = $Delta->VERTICES->minor(All,~[0]);
$edge_indices = $Delta->VERTICES_IN_FACETS;
@edges = map {new Pair<Vector, Vector>([$vertices->row($_->[0]), $vertices->row($_->[1])])} @$edge_indices;

$betas = new Map<Pair<Vector,Vector>,Vector>;
$betas -> {$edges[0]} = new Vector([0,1,1]);
$betas -> {$edges[1]} = new Vector([0,0,1]);
$betas -> {$edges[2]} = new Vector([0,0,0,0,1]);
$betas -> {$edges[3]} = new Vector([3]);

$gamma = new Matrix([[0,5], [0,3], [0,0], [1,2], [2,5], [3,5], [3,0]]);

Click here for additional output 
 
polymake: used package cdd
 cddlib
 Implementation of the double description method of Motzkin et al.
 Copyright by Komei Fukuda.
 http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/

polymake: used package ppl
 The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
 and other numerical abstractions.
 http://www.cs.unipr.it/ppl/

In [13]:
print multiplicity(0, $Delta, $betas, $gamma, true); print "\n";

1440


In [ ]:
print count_lattice_paths_with_multiplicity(0, $Delta, $betas);